In [3]:
import pandas as pd
import numpy as np

In [4]:
#Step 1: Read Data
from io import StringIO

df1=pd.DataFrame({'A':['A0','A1','A2','A3'],'B':['B0','B1','B2',np.nan]})

# Sample CSV data as a string
data = StringIO("""
Order ID,Date,Customer ID,Region,Product,Category,Quantity,Unit Price,Discount,Total Price
1001,2023-01-05,C001,West,Notebook,Stationery,2,100.0,0.05,190.0
1002,2023-01-07,C002,East,Pen,Stationery,10,10.0,0.0,100.0
1003,2023-01-08,C001,West,Pencil,Stationery,5,5.0,0.1,22.5
1004,2023-01-10,C003,South,Mouse,Electronics,1,500.0,0.2,400.0
1005,2023-01-12,C004,North,Keyboard,Electronics,1,800.0,0.1,720.0
""")

# Read the data into a pandas DataFrame
df = pd.read_csv(data)

In [11]:
#Step 3.1 - snapshot of data
df.head()
#Step 3.2 - Column and Info
df.info()
#Step 3.3 - Summary Statistics
df.describe()
#Step 3.4 - Number of Rows and columns
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Order ID     5 non-null      int64  
 1   Date         5 non-null      object 
 2   Customer ID  5 non-null      object 
 3   Region       5 non-null      object 
 4   Product      5 non-null      object 
 5   Category     5 non-null      object 
 6   Quantity     5 non-null      int64  
 7   Unit Price   5 non-null      float64
 8   Discount     5 non-null      float64
 9   Total Price  5 non-null      float64
dtypes: float64(3), int64(2), object(5)
memory usage: 532.0+ bytes


(5, 10)

In [59]:
#Step 4
#4.1 - Date column to datetime format, currently its in object format
df['Date']=pd.to_datetime(df['Date'])
#4.2 - Check for missing values, .sum() gives count of missing or NA values in each column
df.isnull().sum()
#4.3 - Drop missing values
df.dropna()

,Order ID,Date,Customer ID,Region,Product,Category,Quantity,Unit Price,Discount,Total Price
0,1001,2023-01-05,C001,West,Notebook,Stationery,2,100.0,0.05,190.0
1,1002,2023-01-07,C002,East,Pen,Stationery,10,10.0,0.00,100.0
2,1003,2023-01-08,C001,West,Pencil,Stationery,5,5.0,0.10,22.5
3,1004,2023-01-10,C003,South,Mouse,Electronics,1,500.0,0.20,400.0
4,1005,2023-01-12,C004,North,Keyboard,Electronics,1,800.0,0.10,720.0


In [25]:
df1.isnull().sum()

,0
A,0
B,1


In [34]:
#df1=df1.dropna() --> For dropping whole rows where NA values are there in any column
df1=df1.fillna('B3') # --> For imputing values

In [35]:
df1

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [36]:
df

,Order ID,Date,Customer ID,Region,Product,Category,Quantity,Unit Price,Discount,Total Price
0,1001,2023-01-05,C001,West,Notebook,Stationery,2,100.0,0.05,190.0
1,1002,2023-01-07,C002,East,Pen,Stationery,10,10.0,0.00,100.0
2,1003,2023-01-08,C001,West,Pencil,Stationery,5,5.0,0.10,22.5
3,1004,2023-01-10,C003,South,Mouse,Electronics,1,500.0,0.20,400.0
4,1005,2023-01-12,C004,North,Keyboard,Electronics,1,800.0,0.10,720.0


In [51]:
#Step 5.1 - Using df.loc[] to filter for records
df.loc[df['Region']=='West']
df.loc[(df['Region']=='West')&(df['Product']=='Notebook')]
#Step 5.2 - Using df.iloc[] to slice records
df.iloc[:,0:3] #- First 3 columns
df.iloc[0:3] # - First 3 rows (0,1,2)
#5.3 - Taking out columns by column names
df[['Unit Price','Discount']]

,Unit Price,Discount
0,100.0,0.05
1,10.0,0.00
2,5.0,0.10
3,500.0,0.20
4,800.0,0.10


In [13]:
#Step 6 - Sorting, Ranking, create new columns
#6.1
df=df.sort_values('Total Price',ascending=False)
#6.2
df['Revenue']=df['Quantity']*df['Unit Price']*(1-df['Discount'])
df['Profit']=df['Revenue']-(df['Quantity']*df['Unit Price']*0.6)
df['Discount Type'] = df['Discount'].apply(lambda X: "High" if X>=0.1 else "Low")
#df
df=df.drop('Discount Type',axis=1)
#df

In [7]:
#Step 7: Group By and other aggregations
df.groupby('Region')['Total Price'].sum()

,Total Price
Region,
East,100.0
North,720.0
South,400.0
West,212.5


In [9]:
df.groupby(['Region','Category'])['Total Price'].sum()

,,Total Price
Region,Category,
East,Stationery,100.0
North,Electronics,720.0
South,Electronics,400.0
West,Stationery,212.5


In [11]:
df.groupby('Product')['Discount'].mean()

,Discount
Product,
Keyboard,0.10
Mouse,0.20
Notebook,0.05
Pen,0.00
Pencil,0.10


In [14]:
df.groupby('Region').agg({'Revenue':'sum','Discount':'mean'})

,Revenue,Discount
Region,,
East,100.0,0.000
North,720.0,0.100
South,400.0,0.200
West,212.5,0.075


In [15]:
pd.pivot_table(df, index = 'Region', columns = 'Category', values = 'Revenue', aggfunc = 'sum')

Category,Electronics,Stationery
Region,,
East,NaN,100.0
North,720.0,NaN
South,400.0,NaN
West,NaN,212.5


In [17]:
import pandas as pd

customers = pd.DataFrame({
    'Customer ID': ['C001', 'C002', 'C003', 'C004'],
    'Customer Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Email': ['alice@mail.com', 'bob@mail.com', 'charlie@mail.com', 'david@mail.com']
})

In [20]:
df=pd.merge(df,customers, on= 'Customer ID', how='inner')

In [21]:
df['Date'] = pd.to_datetime(df['Date'])

In [25]:
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.day_name()
df

,Order ID,Date,Customer ID,Region,Product,Category,Quantity,Unit Price,Discount,Total Price,Revenue,Profit,Customer Name,Email,Month,Weekday
0,1005,2023-01-12,C004,North,Keyboard,Electronics,1,800.0,0.10,720.0,720.0,240.0,David,david@mail.com,1,Thursday
1,1004,2023-01-10,C003,South,Mouse,Electronics,1,500.0,0.20,400.0,400.0,100.0,Charlie,charlie@mail.com,1,Tuesday
2,1001,2023-01-05,C001,West,Notebook,Stationery,2,100.0,0.05,190.0,190.0,70.0,Alice,alice@mail.com,1,Thursday
3,1002,2023-01-07,C002,East,Pen,Stationery,10,10.0,0.00,100.0,100.0,40.0,Bob,bob@mail.com,1,Saturday
4,1003,2023-01-08,C001,West,Pencil,Stationery,5,5.0,0.10,22.5,22.5,7.5,Alice,alice@mail.com,1,Sunday


In [26]:
df[df['Date'] > '2023-01-08']

,Order ID,Date,Customer ID,Region,Product,Category,Quantity,Unit Price,Discount,Total Price,Revenue,Profit,Customer Name,Email,Month,Weekday
0,1005,2023-01-12,C004,North,Keyboard,Electronics,1,800.0,0.1,720.0,720.0,240.0,David,david@mail.com,1,Thursday
1,1004,2023-01-10,C003,South,Mouse,Electronics,1,500.0,0.2,400.0,400.0,100.0,Charlie,charlie@mail.com,1,Tuesday


In [27]:
df['Product'].value_counts()

,count
Product,
Keyboard,1
Mouse,1
Notebook,1
Pen,1
Pencil,1


In [30]:
df['Region'].nunique()
#unique() - Gives you the distinct values
#nunique() - Gives you the count of such values

4

In [31]:
df.to_csv('cleaned_sales_data.csv',index=False)

In [32]:
import os
os.getcwd()

'/content'